# Dependancies

In [97]:
import numpy as np
import pandas as pd
import networkx as nx
%matplotlib inline
import matplotlib.pyplot as plt
import datetime as dt
from IPython.display import Image
import sqlite3
from networkx.algorithms import community
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import clear_output
# from graphviz import Digraph
# import graphviz
# import pydot
import os
import sys
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

try:
    import tkinter.ttk
    from tkinter import *
#     from tkinter.ttk import *
except:
    from tkinter import *

from tkinter import messagebox


# Class

## The scan class

In [98]:
class Emr_Scan():
    
    now=dt.date.today().strftime('%Y-%m-%d')
    time_of_day=dt.datetime.today().strftime('%I:%M:%S %p')
    sqlite_server=r'\\camradfs01.ccc.coopcam.com\testlab_i\CTDC_Systems\HCTC_EMR_CSP_database\emr_database.db'
    conn=sqlite3.connect(sqlite_server)
    cur=conn.cursor()
    
    
    def __init__(self,scan_string):
        self.scan_string=scan_string
        self.query_database()
        
    def query_database(self):
        """query the database view which contains the equipment ID, classification, model number and s/n"""
        
        self.the_sql_command=f"SELECT * FROM emr_equip_ID_table WHERE EQUIPMENT_ID='{self.scan_string}'"
        
        print(self.the_sql_command)
        
        self.df1=pd.read_sql_query(sql=self.the_sql_command,con=Emr_Scan.conn, index_col=None)
        
        self.TLF_mode_list()
        
        return self.df1
    
    def TLF_mode_list(self):
        """generate the Top Level Failure Mode List from querying the Malfunction Failure Mode for the specified equipment"""
        self.equip_class=self.df1['EQUIPMENT_CLASSIFICATION'].values[0]
        self.the_sql_command=f"select DISTINCT emr_data_1.MALFUNCTION_FAILURE_MODE from emr_data_1 where EQUIPMENT_CLASSIFICATION='{self.equip_class}'";
        
        print(self.equip_class)
        
        print (self.the_sql_command)
        
        self.TLF=pd.read_sql_query(sql=self.the_sql_command,con=Emr_Scan.conn, index_col=None)
        
        self.TLF_list=list(self.TLF['MALFUNCTION_FAILURE_MODE'])
        
        return self.TLF_list
    
    
    def upload_emr(self,location,reported_by,tlfm,describe):
        """upload (INSERT) user's input into the emr_queue database table"""
        
        columns=('Date_Reported','Time_of_Day','Location','Reported_By','Equipment_ID','Equipment_Class','Observed_Failure_Mode','Description')
        v=(self.now,self.time_of_day,location,reported_by,self.scan_string,self.equip_class,tlfm,describe)
        
        self.the_sql_command=f"INSERT INTO emr_queue {columns} VALUES {v};"
        Emr_Scan.cur.execute(self.the_sql_command)
        Emr_Scan.conn.commit()
        
        return self.the_sql_command
    
    

## the interface class

### tkinter window

In [99]:
class My_Window(Tk):
    
    now=dt.date.today().strftime('%B %d, %Y')
    time_of_day=dt.datetime.today().strftime('%I:%M:%S %p')

    def __init__(self,parent,*args,**kargs):
        Tk.__init__(self,parent,*args,**kargs)
        self.parent=parent
        self.initialize()
        
        self.banner=Label(self,text=f'Equipment Malfunction Report Form',fg='white',bg='blue',font='Ariel 30 bold')
        self.banner.grid(row=0,column=0)
        self.date_banner=Label(self,text=f'{My_Window.now}',fg='white',bg='blue',font='Ariel 20 bold')
        self.date_banner.grid(row=1,column=0)
        
        
    def initialize(self):
        self.title('EMR SUBMISSION FORM')
#         the_window_width=self.winfo_screenwidth()
#         the_window_height=self.winfo_screenheight()
#         self.configure(width=the_window_width,height=the_window_height)
#         the_window_width=1200
#         the_window_height=700
#         self.geometry(f'{the_window_width}x{the_window_height}+0+0')
#         self.attributes('-fullscreen', False)
        self['borderwidth']=4
        self['bg']='blue' 
        
    
        

### tkinter frame

In [100]:
class The_Frame(Frame):
    
        
    def __init__ (self,the_window,*args,**kwargs):
        """Instance of the frame"""
        Frame.__init__(self,the_window,*args,**kwargs)
    
        self.frame=Frame(self)
        self['background']='red'
        self['relief']='raised'
        self['borderwidth']=9
        self.grid(row=2,column=0)
        banner_text='Scan equipment barcode. Select the applicable failure mode and provide a description'
        self.frame_banner=Label(self,text=banner_text,fg='white',bg='red',font='Ariel 15 bold')
        self.frame_banner.grid(row=0,column=0,columnspan=5,pady=15)
        
        self.widget_frame()
        self.the_list_box()
        self.frame_text()
        self.buttons()
        self.location_drop_down()
        self.reported_by()
        

    def widget_frame(self):
        self.wf=Frame(self,bg='blue',relief='sunken')
        self.wf['borderwidth']=9
        self.wf.grid(row=2,column=1,columnspan=1,sticky=W)

    def the_list_box(self):
        """Creates list box to list the failure per the equipment class"""
        self.listbox_banner=Label(self,text=f'FAILURE MODE',fg='white',bg='red',font='Ariel 10 bold')
        self.listbox_banner.grid(row=1,column=0,columnspan=1)
        
        self.list_box=Listbox(self,font=8,bg='yellow',borderwidth=2,height=20,width=30)
        self.list_box.grid(row=2,column=0,columnspan=1,sticky=W)
        
   
    def submit(self):
        """Uploads EMR to the emr_queue"""
        
        if (self.location_var.get()=='' or self.entry_var.get()=='' or self.reported_by=='' or self.describe.get("1.0","end-1c")==''):
            messagebox.showinfo("EMR NOT SUBMITTED", "COMPLETE ALL FIELDS BEFORE SUBMITTING THE EMR")
            return
        
        
        the_date=self.now=dt.date.today().strftime('%Y-%m-%d')
        the_time=self.time_of_day=dt.datetime.today().strftime('%I:%M:%S %p')
        
        equip_id=self.entry_var.get()
        reported_by=self.technician.get()
        the_location=self.loc_combo.get()
        failure_mode=self.list_box.get(ANCHOR)
        tech_describe=self.describe.get("1.0","end-1c")
        
        if (failure_mode==''):
            failure_mode='To Be Determined'
        
        print(the_date)
        print(the_time)
        print(equip_id)
        print(reported_by)
        print(the_location)
        print(failure_mode)
        print(tech_describe)
        print(self.e_class)
        
        
        columns=('Date_Reported','Time_of_Day','Location','Reported_By','Equipment_ID',
                 'Equipment_Class','Observed_Failure_Mode','Description')
        v=(the_date,the_time,the_location,reported_by,equip_id,self.e_class,failure_mode,tech_describe)
        
        sqlite_server=r'\\camradfs01.ccc.coopcam.com\testlab_i\CTDC_Systems\HCTC_EMR_CSP_database\emr_database.db'
        try:
            conn=sqlite3.connect(sqlite_server)
            cur=conn.cursor()
            self.the_sql_command=f"INSERT INTO emr_queue {columns} VALUES {v};"
            cur.execute(self.the_sql_command)
            conn.commit()
            messagebox.showinfo("Title", "EMR HAS BEEN SUBMITTED")
        except:
            messagebox.showinfo("Title", "UNABLE TO ACCESS DATABASE. TRY AGAIN OR CONTACT SYSTEMS GROUP.")
     
    def get_TLFM(self):
        """Generates the Top Level Failure Mode (TLFM) for the equipment class."""
        
#         self.equip_scan.set(self.entry_var)
        self.equip_scan.focus_set()
        print(f'cat\n{self.entry_var.get()}')
        a=Emr_Scan(self.entry_var.get())
        self.tlfm_list=a.TLF_mode_list()
        self.e_class=a.equip_class
        print(self.tlfm_list)
        self.populate_list_box()
    
    def populate_list_box(self):
        """inserts the TLFM list items into the list box"""
        for item in self.tlfm_list:
            self.list_box.insert(END, item)
         
    def clear_scan_entry(self):
        """Clear all fields in the event of an entry error."""
        self.list_box.destroy()
        self.the_list_box()
        
        self.loc_combo.destroy()
        self.location_drop_down()
        
        self.technician.destroy()
        self.reported_by()
        
        self.equip_scan.destroy()
        self.describe.destroy
        self.frame_text()
        
        
    @staticmethod
    def get_location_list():
        """Queries the database for a distint list of locations."""
        sqlite_server=r'\\camradfs01.ccc.coopcam.com\testlab_i\CTDC_Systems\HCTC_EMR_CSP_database\emr_database.db'
        conn=sqlite3.connect(sqlite_server)
        cur=conn.cursor()
        
        the_sql_command=f'select DISTINCT Location from emr_data_1 ORDER by Location; '
        df1=pd.read_sql_query(sql=the_sql_command,con=conn, index_col=None)
#         print(df1)
        
        gl_list=list(df1['LOCATION'])
        
        return gl_list
        
        
    def location_drop_down(self):
        """The cdrop down box containing the list of locations"""
        c_list=The_Frame.get_location_list()
#         print(c_list)
        self.loc_label=Label(self.wf,text=f'Location',fg='white',bg='blue',font='Ariel 10 bold')
        self.loc_label.grid(row=1,column=3,columnspan=1)
        
        self.location_var=StringVar()
        self.loc_combo=ttk.Combobox(self.wf,font='Ariel 12 bold',width=30,
                                    background='red',textvariable=self.location_var,values=c_list)
        self.loc_combo.grid(row=2,column=3,columnspan=1,sticky=N)
        
    def reported_by(self):
        """Entry field for the user to enter name or initials."""
        self.tech_label=Label(self.wf,text=f'Reported By',fg='white',bg='blue',font='Ariel 10 bold')
        self.tech_label.grid(row=1,column=4,columnspan=1)
        self.tech_var=StringVar()
        self.technician=Entry(self.wf,font='Ariel 12 bold',width=20,textvariable=self.tech_var)
        self.technician.grid(row=2,column=4,columnspan=1,sticky=N,padx=10)
        self.technician.focus_set()
        
    def frame_text(self):
        
        self.scan_label=Label(self.wf,text=f'Equipment ID',fg='white',bg='blue',font='Ariel 10 bold')
        self.scan_label.grid(row=1,column=1,columnspan=1)
        
#         self.equip_scan=Text(self,borderwidth=2,height=2,width=30,font=15,wrap=WORD)
#         self.equip_scan.insert(INSERT,"Scan Equipment")
        self.entry_var=StringVar()
        self.equip_scan=Entry(self.wf,font='Ariel 12 bold',width=30,textvariable=self.entry_var)
        self.equip_scan.grid(row=2,column=1,columnspan=1,sticky=N,padx=10)
        self.equip_scan.focus_set()
        
#         instruct_words='Scan equipment bar code'
#         self.scan_instruct=Label(self,text=f'{instruct_words}',fg='white',bg='red',font='Ariel 10 italic')
#         self.scan_instruct.grid(row=3,column=1,columnspan=1,sticky=N)
        
   
#         p=self.equip_scan.get("1.0","end-1c")
        
        self.describe_label=Label(self.wf,text=f'Describe the Incident',fg='white',bg='blue',font='Ariel 10 bold')
        self.describe_label.grid(row=3,column=1,columnspan=2,pady=10)
        
        self.describe=Text(self.wf,borderwidth=2,bg='yellow',height=10,width=30,font=15,wrap=WORD)
        self.describe.grid(row=4,column=0,columnspan=2,sticky=N)
        
    def exit_program(self):
        """Exit program"""
        My_Window.bye_bye(self)
        
        
        
    def buttons(self):
        """Buttons for actions"""
        text='TLFM'
        self.the_button=Button(self,text=text,bg='black',fg='white',font='Ariel 12 bold',relief='raised',command=self.get_TLFM)
        self.the_button.grid(row=3,column=0,sticky=W)
        
        tlfm_words='Click the TLFM button to get a list of possible Failure Modes.'
        self.tlfm_words=Label(self,text=tlfm_words,fg='yellow',bg='red',font='Ariel 12 bold')
        self.tlfm_words.grid(row=3, column=0,columnspan=2,padx=20)
        
        text='Clear'
        self.the_button=Button(self.wf,text=text,bg='white',fg='blue',font='Ariel 10 bold',relief='raised',command=self.clear_scan_entry)
        self.the_button.grid(row=4,column=4,sticky=S)
        
        text='Submit'
        self.the_button=Button(self.wf,text=text,bg='green',fg='yellow',font='Ariel 10 bold',relief='raised',command=self.submit)
        self.the_button.grid(row=4,column=5,sticky=S)
        
        text='Exit'
        self.the_button=Button(self,text=text,bg='black',fg='yellow',font='Ariel 10 bold',relief='raised',command=lambda: bye_bye())
        self.the_button.grid(row=4,column=3,sticky=S)
        
          
    
#     def __str__(self):
#         print(self.equip_scan.get("1.0","end-1c"))
#         return print(self.equip_scan.get("1.0","end-1c"))
        
    

# Functions

In [101]:
def bye_bye():
    root.destroy()

# Main

## User Interface

In [102]:
root=My_Window(None)

In [103]:
form=The_Frame(root)

In [104]:
mainloop()

cat
Bemco Unit -3
SELECT * FROM emr_equip_ID_table WHERE EQUIPMENT_ID='Bemco Unit -3'
Temperature Control Unit
select DISTINCT emr_data_1.MALFUNCTION_FAILURE_MODE from emr_data_1 where EQUIPMENT_CLASSIFICATION='Temperature Control Unit'
Temperature Control Unit
select DISTINCT emr_data_1.MALFUNCTION_FAILURE_MODE from emr_data_1 where EQUIPMENT_CLASSIFICATION='Temperature Control Unit'
['TCU Failed Heating', 'TBD', 'No 480VAC power', 'TCU Failed Cooling', 'Scheduled Maintenance\n', 'TCU Failed Defrost', 'No Communication (RS-485)', 'No LN2 Source', 'No 120VAC power', 'TCU Vibration', 'None', 'No communication (PC to TCU)', 'Incorrect Sensor Reading', 'Software Anomaly', 'Software Error', 'Electrical (Controls)']
2019-10-11
01:57:08 PM
Bemco Unit -3
Jim Dowd
Cell 15
TCU Failed Heating
Unit stopped heating 2 hours into the test. 
Temperature Control Unit
